### 멜론 차트

In [1]:
import requests
import pandas as pd 
from bs4 import BeautifulSoup

- 데이터 가져오기

In [2]:
url = 'https://www.melon.com/chart/index.htm'
res = requests.get(url)
res.text

''

In [3]:
# Chrome user agent
header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
            AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}
res = requests.get(url, headers=header)
res.text[:100]

'<!DOCTYPE html>\r\n<html lang="ko">\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n<head>\r\n\t\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\r\n\t'

In [4]:
soup = BeautifulSoup(res.text, 'html.parser')

- 2. 원하는 데이터 찾기

In [5]:
date_str = soup.select_one('.yyyymmdd').get_text().strip() + \
                soup.select_one('.hhmm').get_text().strip()
date_str = date_str.replace('.', '').replace(':', '')
date_str

'202309181300'

In [6]:
len(soup.select('.lst100'))

50

In [7]:
trs = soup.select('.lst50') + soup.select('.lst100')
len(trs)

100

- 3. 하나를 선택해서 원하는 정보 추출

In [8]:
tr = trs[0]
rank = int(tr.select_one('.rank').get_text().strip())
rank

1

In [9]:
title = tr.select_one('.ellipsis.rank01').get_text().strip()
title

'Love Lee'

In [10]:
artist = tr.select_one('.ellipsis.rank02 > a').get_text().strip()
artist

'AKMU (악뮤)'

In [11]:
album = tr.select_one('.ellipsis.rank03').get_text().strip()
album

'Love Lee'

In [12]:
# 좋아요는 BeautifulSoup로는 가져올 수 없음
like = tr.select_one('.button_etc.like > span.cnt').get_text()
like

'\n총건수\r\n\t\t\t\t\t\t\t\t\t\t\t\t0\r\n\t\t\t\t\t\t\t\t\t\t\t'

In [13]:
img = tr.select_one('.image_typeAll > img')['src']
img

'https://cdnimg.melon.co.kr/cm2/album/images/113/09/190/11309190_20230818161008_500.jpg/melon/resize/120/quality/80/optimize'

- 한 페이지에 있는 정보 가져오기

In [14]:
data = []
for tr in trs:
    rank = int(tr.select_one('.rank').get_text().strip())
    title = tr.select_one('.ellipsis.rank01').get_text().strip()
    artist = tr.select_one('.ellipsis.rank02 > a').get_text().strip()
    album = tr.select_one('.ellipsis.rank03').get_text().strip()
    data.append({'rank': rank, 'title': title, 'artist': artist, 'album': album})

- 5. 에러 처리
- 6. 모든 페이지
- 7. 데이터프레임 만들고 저장하기

In [15]:
df = pd.DataFrame(data)
df.head()

,rank,title,artist,album
0,1,Love Lee,AKMU (악뮤),Love Lee
1,2,Seven (feat. Latto) - Clean Ver.,정국,Seven (feat. Latto) - Clean Ver.
2,3,"Smoke (Prod. Dynamicduo, Padi)",다이나믹 듀오,스트릿 우먼 파이터2(SWF2) 계급미션
3,4,후라이의 꿈,AKMU (악뮤),Love Lee
4,5,Super Shy,NewJeans,NewJeans 2nd EP 'Get Up'


In [19]:
df.to_csv(f'data/melon_{date_str}.csv', index=False)
pd.read_csv(f'data/melon_{date_str}.csv').tail()

,rank,title,artist,album
95,96,Yogurt Shake,NCT DREAM,ISTJ - The 3rd Album
96,97,That's Hilarious,Charlie Puth,CHARLIE
97,98,In Bloom,ZEROBASEONE (제로베이스원),YOUTH IN THE SHADE
98,99,주저하는 연인들을 위해,잔나비,전설
99,100,TOMBOY,(여자)아이들,I NEVER DIE
